# Introduction 
Building a linear regression model using neural networks

In [54]:
!uv pip install numpy torch lightgbm seaborn matplotlib scikit-learn

Audited 6 packages in 53ms


In [55]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import torch.optim as optim
import lightgbm as lgb
# import xgboost

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import absolute

In [56]:
train = pd.read_csv("data/train.csv")
# assigning data and target
target = torch.from_numpy(train["FloodProbability"].values)
data = torch.from_numpy(train.drop(["FloodProbability", "id"], axis=1).values)

X, y = data, target
n_samples, n_features = X.shape

# data splitting
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)

In [57]:
X_train = X_train.float()
y_train = y_train.float()
X_test = X_test.float()
y_test = y_test.float()

# Create DataLoader for mini-batch training
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 50  # Adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [58]:
# creating a model
class MLP_Regressor(nn.Module):
    """
    Multilayer Perceptron for regression.
    We have 20 features
    """

    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(20, 32), nn.ReLU(), nn.Linear(32, 32), nn.ReLU(), nn.Linear(32, 1)
        )

    def forward(self, x):
        """
        Forward propagation
        """
        return self.layers(x)

In [59]:
# custom loss function since I want to keep the loss metric consistent between my attempt and the kaggle check
class R2Loss(nn.Module):
    def __init__(self):
        super(R2Loss, self).__init__()

    def forward(self, y_pred, y_true):
        ss_tot = torch.sum((y_true - torch.mean(y_true)) ** 2)
        ss_res = torch.sum((y_true - y_pred) ** 2)
        r2 = 1 - ss_res / ss_tot
        return r2

In [60]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize the MLP
mlp = MLP_Regressor()
model_1 = MLP_Regressor().to(device)

# Define the loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# X_train = X_train.to(device)
# X_test = X_test.to(device)
# y_train = y_train.to(device)
# y_test = y_test.to(device)

In [63]:
# Set the number of epochs
epochs = 100

# Training and testing loop
for epoch in range(epochs):
    # Training
    model_1.train()
    train_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        # Move data to the correct device
        inputs, targets = data
        # inputs, targets = inputs.float(), targets.float()
        inputs, targets = inputs.to(device), targets.to(device)

        targets = targets.unsqueeze(1)
        # Forward pass
        y_pred = model_1(inputs)

        # Calculate loss
        loss = loss_fn(y_pred, targets)

        # Zero grad optimizer
        optimizer.zero_grad()

        # Loss backward
        loss.backward()

        # Step the optimizer
        optimizer.step()

        train_loss += loss.item()
        # if i % 10 == 0:
        # print("Loss after mini-batch %5d: %.3f" % (i + 1, train_loss / (i + 1)))
        # train_loss = 0.0
    print(f"train loss: {train_loss/len(train_loader)}")
    train_loss = 0.0

    # train_loss += loss.item() * inputs.size(0)

    # train_loss /= len(train_loader.dataset)

    # ### Testing
    # model_1.eval()
    # test_loss = 0.0
    # with torch.no_grad():
    #     for inputs, targets in test_loader:
    #         # Move data to the correct device
    #         inputs, targets = inputs.to(device), targets.to(device)
    #         targets = targets.unsqueeze(1)
    #         # Forward pass
    #         test_pred = model_1(inputs)

    #         # Calculate loss
    #         loss = loss_fn(test_pred, targets)
    #         test_loss += loss.item() * inputs.size(0)

    # test_loss /= len(test_loader.dataset)

    # if epoch % 1 == 0:
    #     print(
    #         f"Epoch: {epoch + 1} | Train loss: {train_loss:.4f} | Test loss: {test_loss:.4f}"
    #     )

    # Optionally clear cache if using GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

train loss: 0.4301541575292302
train loss: 0.4301546754112237
train loss: 0.4301544646024437
train loss: 0.4301566097782075
train loss: 0.43015926345443095
train loss: 0.4301551126663395
train loss: 0.430154228346399
train loss: 0.4301551173212863
train loss: 0.43015504023476714
train loss: 0.43015163916967847
train loss: 0.43015601075785853
train loss: 0.430155326100816
train loss: 0.43015756396733373
train loss: 0.4301538716831739
train loss: 0.4301512080422987


KeyboardInterrupt: 

In [ ]:
from pprint import pprint

print("The model learned the following values for weights and bias:")
pprint(model_1.state_dict())
# print("\nAnd the original values for weights and bias are:")
# print(f"weights: {weight}, bias: {bias}")

The model learned the following values for weights and bias:
OrderedDict([('layers.0.weight',
              tensor([[ 0.0174, -0.0439,  0.0675, -0.1616, -0.0335, -0.0080, -0.1872,  0.1829,
          0.0683,  0.0632, -0.1320,  0.1208,  0.1210,  0.0828,  0.1114,  0.1583,
          0.0076,  0.1274,  0.1629,  0.1641],
        [ 0.1407, -0.0117, -0.2006, -0.1308,  0.0578,  0.0321,  0.1988, -0.0050,
         -0.2047,  0.1641, -0.0992, -0.2001,  0.0563, -0.0228,  0.0564, -0.1967,
          0.0754, -0.1655, -0.1286, -0.0569],
        [ 0.1668,  0.0870,  0.1492, -0.1167,  0.0720, -0.1533, -0.1104,  0.0074,
          0.1824, -0.0636, -0.2113,  0.0464,  0.2201, -0.1786,  0.2176, -0.0805,
          0.0135, -0.0553, -0.0315,  0.0071],
        [ 0.0596,  0.0144,  0.0861, -0.0594,  0.1180, -0.0905, -0.0499,  0.0715,
         -0.0308,  0.1695,  0.1934,  0.2105, -0.2069,  0.1890, -0.0545, -0.0995,
          0.2139, -0.0033, -0.1882,  0.0198],
        [-0.1343,  0.1958,  0.1732, -0.0338, -0.1875, -0.054

In [ ]:
test = pd.read_csv("data/test.csv")
test_cleaned = test.drop(["id"], axis=1)
test_cleaned = torch.from_numpy(test_cleaned.values)
test_cleaned = test_cleaned.float()

In [ ]:
# test = pd.read_csv("data/test.csv")
with torch.inference_mode():
    y_preds = model_1(test_cleaned)
y_preds

df = pd.DataFrame(y_preds.numpy())

df1 = pd.merge(test["id"], df, left_index=True, right_index=True)

In [ ]:
df1.rename(columns={"O": "FloodProbability"}, inplace=True)

In [ ]:
column_names = df1.columns.tolist()
column_names[1] = "FloodProbability"
df1.columns = column_names

In [ ]:
df1

,id,FloodProbability
0,1117957,0.237746
1,1117958,0.384492
2,1117959,0.345836
3,1117960,0.163628
4,1117961,0.332582
...,...,...
745300,1863257,0.384192
745301,1863258,0.223839
745302,1863259,0.423804
745303,1863260,0.597693


In [ ]:
df1.to_csv("new2_submission.csv", index=False)